# **Loading libraries**

In [ ]:
# Install cuml for GPU processing
!pip install cuml-cu12 --extra-index-url=https://pypi.nvidia.com

Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.2/581.2 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━━━━━ 79.7/200.9 MB 32.6 MB/s eta 0:00:04

In [ ]:
# Loading all necessary libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import TimeSeriesSplit

# Preprocessing
from sklearn.preprocessing import StandardScaler, FunctionTransformer
from sklearn.pipeline import make_pipeline, Pipeline

# Sampling
from sklearn.kernel_approximation import RBFSampler
from sklearn.calibration import CalibratedClassifierCV

# Import models
from sklearn.dummy import DummyClassifier
from cuml.svm import SVC
from cuml.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression, PoissonRegressor
from sklearn.linear_model import SGDClassifier

# Import evaluation metrics
from sklearn.metrics import (accuracy_score, precision_score,
                            recall_score, f1_score, confusion_matrix,
                            classification_report, roc_curve, auc, roc_auc_score)
from sklearn.model_selection import cross_validate, cross_val_predict




# Random Seed
seed = 42

# **Loading the Dataset**

In [ ]:
# Load dataset
url='https://drive.google.com/file/d/1eyqgzDSDuy6n1JZUGodAsiGUTFCzyy2B/view?usp=drive_link'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.head()




# **Data Exploration and Preprocessing for Modeling**

In [ ]:
# Checking the overall specifications of the data
df.info()
df_copy=df



In [ ]:
# Adding month number so we can try and observe if there are months with more goals scored
df['Date'] = pd.to_datetime(df['Date'], format='%Y-%m-%d')
df['Month'] = df['Date'].dt.month


In [ ]:
df.iloc[250]

In [ ]:
# Removing initial n number of rows which do not have average values

## Creating criteria for rows that are going to be removed from our work

rolling_features = [
    'avg_goals_scored_home_5', 'avg_goals_conceded_home_5',
    'avg_goals_scored_away_5', 'avg_goals_conceded_away_5',
    'avg_total_goals_home_5', 'avg_total_goals_away_5',
    'win_rate_home_5', 'win_rate_away_5',
    'Over2.5_home_5', 'Over2.5_away_5'
]


# Count how many rolling features are zero or NaN per row
zero_or_nan_mask = df[rolling_features].apply(lambda row: ((row == 0) | (row.isna())).sum(), axis=1)

# Defining a threshold - NOTE: this is a hard threshold, we could potentially change it to be more flexible based on percentiles
threshold = 5



df_cleaned = df[zero_or_nan_mask < threshold]

In [ ]:
# Number of rows removed

len(df)-len(df_cleaned)

In [ ]:
# Checking the distributions of our variables

df_cleaned.hist(figsize=(15, 10), bins=30)
plt.tight_layout()
plt.show()


As we can see, most of the important features have a normal-ish distribution with some of them being a bit positively skewed (notably the avg_goals variables).

In [ ]:
# Let us do a square root transformation for the avg_goals variables since they tend to have more of a poisson distribution

df_trans = df_cleaned.copy()



In [ ]:
poisson_features = ['avg_goals_scored_home_5', 'avg_goals_conceded_home_5',
    'avg_goals_scored_away_5', 'avg_goals_conceded_away_5',
    'avg_total_goals_home_5', 'avg_total_goals_away_5']

df_trans[poisson_features] = np.sqrt(df_trans[poisson_features])





In [ ]:
# Checking distributions after transformation
df_trans[poisson_features].hist(figsize=(15, 10), bins=30)
plt.tight_layout()
plt.show()

These are still not perfect normal distributions, but it is **close enough** so it will not bother our models too much

Since our dataset consists of time series data, we will use TimeSeriesSplit for cross-validation in this case.

Important to note is that we are using the models on the whole dataset and thus creating General predictive models for all leagues. This approach significantly reduces the computational complexity associated with training and tuning multiple models individually for each nation

Furthermore after creation of this general model, it allows much quicker deployment for any new datasets that might be getting released in the future as it is much more generazible and flexible, as it learns patterns across diverse leagues and countries, making it better suited for handling variations in data without requiring extensive retraining.

This is ofcourse under the note that we have limited Computation resources and want to limit computational costs.

In [ ]:
# Time series split n = 10
target = 'Over2.5'
X = df_trans.drop(target, axis=1)
y = df_trans[target]

time_split = TimeSeriesSplit(n_splits=10)



for train_index, test_index in time_split.split(X):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]


In [ ]:
# Timeseries split on not transformed data (for RandomForest)
X1 = df_cleaned.drop(target, axis=1)
y1 = df_cleaned[target]

time_split = TimeSeriesSplit(n_splits=10)



for train_index, test_index in time_split.split(X1):
    X1_train, X1_test = X1.iloc[train_index], X1.iloc[test_index]
    y1_train, y1_test = y1.iloc[train_index], y1.iloc[test_index]

# **Modeling**
Testing out multiple models:

* Dummy Classifier
* Random Forest
* SVM with Stochastic Gradient Descent
* SVM (and the Kernels)
* Logistic Classification
* XGBoost


The reasons for our choice of machine learning models are as follows:

Random forest was chosen as a sort of better than Dummy Classifier basic model to see how far we could get with the power of Random Forests.

SGDC due to being ideal for large datasets and it allows us to incorporate regularization techniques to prevent overfitting.

SVM RBF and polynomial kernels were chosen because they are strong for capturing complex, non-linear decision boundaries that may come from patterns in match statistics. These kernels allow the model to adapt to subtle variations across different leagues.

Logistic regression for its strong baseline for binary classification.

And finally XGBoost which just generally excels in handling structured data and capturing feature interactions while maintaining high predictive performance and efficiency. Its regularization capabilities also help mitigate overfitting.

In [ ]:
# Dummy Classifer
dummy = DummyClassifier(strategy='most_frequent')

# Random Forest Classifier
forest = RandomForestClassifier(n_estimators=500, random_state=seed, max_features='sqrt', max_depth=5)

In [ ]:
# Linear SVM using SGDC
SGDC_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("calibrated_sgd", CalibratedClassifierCV( # To fit SVM to ROC plot, it must be wrapped in calibrater
        SGDClassifier(max_iter=1000, tol=1e-3,
            random_state=seed,
            class_weight='balanced'
        ),
        cv=5,
        method="sigmoid"
    ))
])



In [ ]:
# SVM with RBF Kernel
rbf_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("calibrated_sgd", CalibratedClassifierCV( # To fit SVM to ROC plot, it must be wrapped in calibrater
        SVC(kernel='rbf'),
        cv=5,
        method="sigmoid"
    ))
])

In [ ]:
# SVM with poly kernel
poly_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("calibrated_sgd", CalibratedClassifierCV( # To fit SVM to ROC plot, it must be wrapped in calibrater
        SVC(kernel='poly'),
        cv=5,
        method="sigmoid"
    ))
])

In [ ]:
# Logistic Regression
log_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(random_state=seed))
])


In [ ]:
# Dropping Date column so models fit properly
X_train = X_train.drop(columns=['Date'])
X_test = X_test.drop(columns=['Date'])


In [ ]:
X1_train = X1_train.drop(columns=['Date'])
X1_test = X1_test.drop(columns=['Date'])

# **Evaluation of Models on the Base Dataset**

In [ ]:
# Creating an overall evaluation function

def evaluate_model(model, X_train, y_train, X_test, y_test):
    # Fit the model
    model.fit(X_train, y_train)

    # Predict on test set
    y_pred = model.predict(X_test)

    # Compute metrics
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)

    # Print results
    print("Test Set Performance:")
    print(f"Accuracy:  {accuracy:.4f}")
    print(f"F1 Score:  {f1:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall:    {recall:.4f}")




Creating Helpful AUC model evaluation function

In [ ]:
def evaluate_model_AUC(model):
    # Get predictions
    y_pred_test = model.predict(X_test)
    y_pred_train = model.predict(X_train)

    # Calculate accuracy
    accuracy_test = accuracy_score(y_test, y_pred_test)
    accuracy_train = accuracy_score(y_train, y_pred_train)

    # Print accuracy
    print(f"Test Accuracy: {accuracy_test}")
    print(f"Train Accuracy: {accuracy_train}")

    # Plot ROC Curve (if model has predict_proba)
    if hasattr(model, 'predict_proba'):
        proba_test = model.predict_proba(X_test)
        proba_train = model.predict_proba(X_train)

        # Safe handling for binary or single-class case
        if proba_test.ndim == 2 and proba_test.shape[1] > 1:
            y_pred_proba_test = proba_test[:, 1]
            y_pred_proba_train = proba_train[:, 1]
        else:
            # If only one column exists, use that
            y_pred_proba_test = proba_test.ravel()
            y_pred_proba_train = proba_train.ravel()

        # Compute ROC and AUC
        fpr, tpr, _ = roc_curve(y_test, y_pred_proba_test)
        roc_auc = auc(fpr, tpr)
        fpr_train, tpr_train, _ = roc_curve(y_train, y_pred_proba_train)
        roc_auc_train = auc(fpr_train, tpr_train)

        # Plot ROC curves
        plt.figure()
        plt.plot(fpr, tpr, label=f'Test ROC curve (area = {roc_auc:.2f})')
        plt.plot(fpr_train, tpr_train, label=f'Train ROC curve (area = {roc_auc_train:.2f})')
        plt.plot([0, 1], [0, 1], 'k--', label='No Skill')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('ROC Curve')
        plt.legend(loc='lower right')
        plt.show()
    else:
        print("ROC curve cannot be plotted as the model does not have predict_proba.")



## Dummy model as a baseline

In [ ]:
# Dummy Performance
evaluate_model(dummy, X_train, y_train, X_test, y_test)

In [ ]:
evaluate_model_AUC(dummy)

## Random Forest

In [ ]:
# Random Forest Performance (Using X1 and y1 as untransformed for Random Forest)
evaluate_model(forest, X1_train, y1_train, X1_test, y1_test)

## SVM stochastic gradient descent

In [ ]:
# SVM SGDC
evaluate_model(SGDC_pipe, X_train, y_train, X_test, y_test)

In [ ]:
evaluate_model_AUC(SGDC_pipe)

## SVM RBF

In [ ]:
# SVM RBF
evaluate_model(rbf_pipe, X_train, y_train, X_test, y_test)

In [ ]:
evaluate_model_AUC(rbf_pipe)

In [ ]:
# SVM Poly
evaluate_model(poly_pipe, X_train, y_train, X_test, y_test)

In [ ]:
evaluate_model_AUC(poly_pipe)

In [ ]:
# Logistic Regression
evaluate_model(log_pipe, X_train, y_train, X_test, y_test)

In [ ]:
evaluate_model_AUC(log_pipe)

# **2. Optimization and tuning for baseline model**


For hyperparemeter tuning we will mainly be using BayesSearch because it is much quicker and practical than both GridSearch and RandomSearch, especially when using GPU on colab, due to the limited runtime on GPUs (disconnections after a few minutes with GridSearch). We know it might not guarantee the very best parameters, but we decided it is the most practical in our scenario.

In [ ]:
!pip install scikit-optimize

In [ ]:
from sklearn.metrics import classification_report
from skopt import BayesSearchCV
from skopt.space import Real, Categorical
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [ ]:
# SGDC Tuning
sgd_search = BayesSearchCV(
    estimator=SGDC_pipe,
    search_spaces=
{
    'calibrated_sgd__estimator__alpha': (1e-5, 1e-2, 'log-uniform'),
    'calibrated_sgd__estimator__loss': ['hinge', 'log_loss'],
    'calibrated_sgd__estimator__penalty': ['l2', 'l1', 'elasticnet']
},

    n_iter=20,
    cv=time_split,
    scoring='accuracy',
    random_state=seed,
    n_jobs=-1
)

sgd_search.fit(X_train, y_train)

# Saving predictions for later evaluation
y_pred_sgd = sgd_search.predict(X_test)


# --- Show the best results ---
print("Best parameters found:", sgd_search.best_params_)
print("Best accuracy score:", sgd_search.best_score_)

**Tuning SVMs (poly and RBF kernels)** Unfortunately we could not get any reasonable runtimes with the tuning of SVMs, one of our collegues even encountered a usage limit with Google Colabs GPU runtime, where the GPU was disabled for that user after running it for tens of minutes, for an undisclosed amount of time.

Due to these constraints, we have not included the tuning results in this report. However, we have left commented code in place to demonstrate the process we had set up for this task.

In [ ]:
# Tuning SVMs

# Creating new pipeline so it fits into the bayes search and tests if either RBF or poly is better for accuracy overall
# svm_pipe = Pipeline([
#    ("scaler", StandardScaler()),
#    ("calibrated_svm", CalibratedClassifierCV(
#        estimator=SVC(random_state=seed),
#        cv=time_split,
#        method="sigmoid"
#    ))
#])

# Bayesian Search
#svm_bayes_search = BayesSearchCV(
#    estimator=svm_pipe,
#    search_spaces= {
#    'calibrated_svm__estimator__C': (0.1, 10.0, 'log-uniform'),
#    'calibrated_svm__estimator__gamma': (1e-3, 1.0, 'log-uniform'),
#    'calibrated_svm__estimator__kernel': ['rbf', 'poly'],
#    'calibrated_svm__estimator__degree': (2, 5)  # only relevant if kernel='poly'
#},
#    n_iter=20,
#    cv=time_split,
#    scoring='accuracy',
#    random_state=42,
#    n_jobs=-1
#)

# Fit
#svm_bayes_search.fit(X_train, y_train)

# Best params and score
#print("Best Parameters:", svm_bayes_search.best_params_)
#print("Best accuracy score:", svm_bayes_search.best_score_)

For Random Forest, we had to resort to using GridSearch, as BayesSearch did not work properly in this case and produced unexpected errors.

Fortunately, the GridSearch run completes relatively quickly, so we decided to stick with this approach for tuning this model.

In [ ]:
# Tuning Random Forest
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV
accuracy_scorer = make_scorer(accuracy_score)


param_grid_forest = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [5, 10, 15, 20]
}

grid_forest = GridSearchCV(forest, param_grid=param_grid_forest, cv=time_split, scoring='accuracy', n_jobs=-1) # Use original X and y, it will be processed by time_series_cv()
grid_forest.fit(X_train,y_train)

# Best params and score
print("Best Parameters:", grid_forest.best_params_)
print("Best accuracy score:", grid_forest.best_score_)

In [ ]:
best_forest = RandomForestClassifier(random_state = seed, max_depth = 5, max_features= 'sqrt', n_estimators= 100)

best_forest.fit(X_train, y_train)
y_pred_forest = best_forest.predict(X_test)
accuracy_score(y_test, y_pred_forest)

In the next cell, we will tune logistic regression model and use Bayesian optimization to find the best regularization parameter and solver.

In [ ]:
# --- Define pipeline ---
log_pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('log_reg', LogisticRegression(random_state=seed, max_iter=200))
])

# --- Define parameter space ---
param_space = {
    'log_reg__C': Real(0.001, 10, prior='log-uniform'),
    'log_reg__penalty': Categorical(['l1', 'l2']),
    'log_reg__solver': Categorical(['liblinear', 'saga'])
}


# --- Define the Bayesian optimizer ---
bayes_search = BayesSearchCV(
    estimator=log_pipe,
    search_spaces=param_space,
    n_iter=20,                  # number of iterations (budget)
    scoring='f1',
    cv=time_split,
    n_jobs=-1,
    random_state=seed,
    verbose=1
)

# --- Fit the optimizer ---
bayes_search.fit(X_train, y_train)

# --- Show the best results ---
print("Best parameters found:", bayes_search.best_params_)
print("Best F1 score:", bayes_search.best_score_)

# --- Evaluate on test set ---
optimized_model = bayes_search.best_estimator_
y_pred = optimized_model.predict(X_test)

print(classification_report(y_test, y_pred))

In [ ]:
lr_optimalized_model = Pipeline([
    ("scaler", StandardScaler()),
    ("log_reg", LogisticRegression(
        C=bayes_search.best_params_["log_reg__C"],
        penalty=bayes_search.best_params_["log_reg__penalty"],
        solver=bayes_search.best_params_["log_reg__solver"], # Use the best solver
        # l1_ratio=bayes_search.best_params_.get("log_reg__l1_ratio"), # Remove l1_ratio as it's not needed with liblinear or l1
        max_iter=500,
        random_state=seed
    ))
])

lr_optimalized_model.fit(X_train, y_train)
y_pred = lr_optimalized_model.predict(X_test)

In [ ]:
evaluate_model_AUC(lr_optimalized_model)

Bayesian optimization unfortunately did not help in this case.

## XGBoost

---
Let us try XGBoost to see if it performs better than the previous models.


In [ ]:
#pip install xgboost optuna
import xgboost as xgb

In [ ]:
# duplicate test and train sets
y_train_xgb = y_train
y_test_xgb = y_test
X_train_xgb = X_train
X_test_xgb = X_test


XGB classifier

In [ ]:
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    max_depth=3,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=seed,
    use_label_encoder=False,
    eval_metric='logloss'
)

xgb_model.fit(X_train_xgb, y_train_xgb)



In [ ]:
evaluate_model_AUC(xgb_model)

XGBoost performs slighty better, but overfits. Let us optimize parameters

### Optimize hyperparameters of XGBoost

Now I´ll try to get better porformance by aplying optuna to find best parameters for the model.

In [ ]:
!pip install bayesian-optimization



In [ ]:
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from bayes_opt import BayesianOptimization
import numpy as np

# Definujeme hledanou funkci
def xgb_evaluate(max_depth, learning_rate, subsample, colsample_bytree, gamma):
    # Převod float -> int pro max_depth
    max_depth = int(max_depth)

    model = XGBClassifier(
        n_estimators=100,
        max_depth=max_depth,
        learning_rate=learning_rate,
        subsample=subsample,
        colsample_bytree=colsample_bytree,
        gamma=gamma,
        objective='binary:logistic',
        eval_metric='logloss',
        use_label_encoder=False,
        random_state=seed
    )

    # 5-fold cross-validation
    score = cross_val_score(model, X_train_xgb, y_train_xgb, cv=5, scoring='roc_auc').mean()
    return score


In [ ]:
# Ranges for optimization
pbounds = {
    'max_depth': (3, 10),
    'learning_rate': (0.01, 0.3),
    'subsample': (0.5, 1.0),
    'colsample_bytree': (0.5, 1.0),
    'gamma': (0, 5)
}


In [ ]:
optimizer = BayesianOptimization(
    f=xgb_evaluate,
    pbounds=pbounds,
    random_state=seed,
    verbose=2
)

optimizer.maximize(
    init_points=5,
    n_iter=25
)

print(optimizer.max)


In [ ]:
best_params = optimizer.max['params']
final_model = XGBClassifier(
    n_estimators=100,
    max_depth=int(best_params['max_depth']),
    learning_rate=best_params['learning_rate'],
    subsample=best_params['subsample'],
    colsample_bytree=best_params['colsample_bytree'],
    gamma=best_params['gamma'],
    objective='binary:logistic',
    eval_metric='logloss',
    use_label_encoder=False,
    random_state=seed
)

final_model.fit(X_train_xgb, y_train_xgb)


In [ ]:
evaluate_model_AUC(final_model)

We added a validation set, applied early stop and try it optuna again with different settings.





In [ ]:
!pip install optuna
import xgboost as xgb
import optuna
from sklearn.metrics import log_loss, accuracy_score, roc_curve, auc
import matplotlib.pyplot as plt

# Split train into train/eval for Optuna tuning
cut = int(len(X_train_xgb) * 0.85)
X_tr, X_eval = X_train_xgb.iloc[:cut], X_train_xgb.iloc[cut:]
y_tr, y_eval = y_train_xgb.iloc[:cut], y_train_xgb.iloc[cut:]

def objective(trial):
    params = {
        'objective': 'binary:logistic',
        'eval_metric': 'logloss',
        'tree_method': 'hist',
        'seed': seed,
        'random_state': seed,
        'eta': trial.suggest_float('learning_rate', 0.01, 0.2, log=True),
        'max_depth': trial.suggest_int('max_depth', 2, 6),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        'subsample': trial.suggest_float('subsample', 0.6, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.5, 1.0),
        'gamma': trial.suggest_float('gamma', 0.0, 2.0),
        'lambda': trial.suggest_float('reg_lambda', 1.0, 50.0, log=True),
        'alpha': trial.suggest_float('reg_alpha', 0.0, 2.0),
        'verbosity': 0,
    }

    dtrain = xgb.DMatrix(X_tr, label=y_tr)
    deval = xgb.DMatrix(X_eval, label=y_eval)

    num_boost_round = 3000
    booster = xgb.train(
        params,
        dtrain,
        num_boost_round=num_boost_round,
        evals=[(deval, 'eval')],
        early_stopping_rounds=100,
        verbose_eval=False
    )

    best_rounds = getattr(booster, 'best_iteration', None)
    if best_rounds is None:
        best_rounds = getattr(booster, 'best_ntree_limit', num_boost_round)

    trial.set_user_attr('best_rounds', int(best_rounds))

    try:
        proba = booster.predict(deval, iteration_range=(0, int(best_rounds)))
    except TypeError:
        proba = booster.predict(deval, ntree_limit=int(best_rounds))

    return float(log_loss(y_eval, proba))

# Run Optuna study
study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=seed))
study.optimize(objective, n_trials=40, show_progress_bar=True)

best_params = study.best_params
best_rounds = int(study.best_trial.user_attrs.get('best_rounds', 300))
print("Best params:", best_params)
print("Best eval logloss:", study.best_value)
print("Best boosting rounds:", best_rounds)

# Train final model on full training set
final_params = {
    'objective': 'binary:logistic',
    'eval_metric': 'logloss',
    'tree_method': 'hist',
    'eta': best_params['learning_rate'],
    'max_depth': best_params['max_depth'],
    'min_child_weight': best_params['min_child_weight'],
    'subsample': best_params['subsample'],
    'colsample_bytree': best_params['colsample_bytree'],
    'gamma': best_params['gamma'],
    'lambda': best_params['reg_lambda'],
    'alpha': best_params['reg_alpha'],
    'verbosity': 0,
}

dfull = xgb.DMatrix(X_train_xgb, label=y_train_xgb)
final_booster = xgb.train(
    final_params,
    dfull,
    num_boost_round=best_rounds,
    verbose_eval=False
)

# Predictions
dtrain = xgb.DMatrix(X_train_xgb)
dtest = xgb.DMatrix(X_test_xgb)

p_train = final_booster.predict(dtrain)
p_test = final_booster.predict(dtest)

y_pred_train = (p_train >= 0.5).astype(int)
y_pred_test = (p_test >= 0.5).astype(int)

# Accuracy
acc_train = accuracy_score(y_train_xgb, y_pred_train)
acc_test = accuracy_score(y_test_xgb, y_pred_test)

print(f"Train Accuracy: {acc_train:.3f}")
print(f"Test  Accuracy: {acc_test:.3f}")

# ROC curves + AUC
fpr_tr, tpr_tr, _ = roc_curve(y_train_xgb, p_train)
fpr_te, tpr_te, _ = roc_curve(y_test_xgb, p_test)
auc_tr = auc(fpr_tr, tpr_tr)
auc_te = auc(fpr_te, tpr_te)

plt.figure()
plt.plot(fpr_te, tpr_te, label=f"Test ROC (AUC={auc_te:.2f})")
plt.plot(fpr_tr, tpr_tr, label=f"Train ROC (AUC={auc_tr:.2f})", alpha=0.6)
plt.plot([0, 1], [0, 1], "k--", label="No Skill")
plt.xlim([0, 1]); plt.ylim([0, 1.05])
plt.xlabel("False Positive Rate"); plt.ylabel("True Positive Rate")
plt.title("ROC Curve — XGBoost Booster")
plt.legend(loc="lower right")
plt.show()


This overfits even more and the accuracy for test set does not rise.

#Best baseline model

After applying various tuning and boosting methods, the best baseline model is SVM stochastic gradient descent with a specific set of hyperparameters with the accuracy of roughly 0.545.

# Performance of the best optimized model for each Country

**Fully Optimized SGDC**

In [ ]:
new_df = pd.DataFrame({
    'y_test': y_test,
    'y_pred_test': y_pred_sgd,
    'Div_enc': X_test['Div_enc'],
})

# Mapping
country_to_code = {
    'Belgium': [0],
    'Germany': [1, 2],
    'England': [3, 4, 5, 6],
    'France': [7, 8],
    'Greece': [9],
    'Italy': [10, 11],
    'Netherland': [12],
    'Portugal': [13],
    'Scotland': [14, 15, 16, 17],
    'Spain': [18, 19],
    'Turkey': [20]
}


# De-Encode
code_to_country = {code: country for country, codes in country_to_code.items() for code in codes}

# Convert your numeric series
new_df["Country"] = new_df["Div_enc"].map(code_to_country)
print(new_df)

In [ ]:
def accuracy(group):
    return pd.Series({'accuracy': accuracy_score(group['y_test'], group['y_pred_test'])})

accuracies = new_df.groupby('Country').apply(accuracy, include_groups=False)
accuracy_report = accuracies.sort_values(by='accuracy', ascending=False)
accuracy_report

As we can see generally the worst performing Nation for this basic data model is Spain and the best is Portugal. With this in mind we can follow through with the Alternative model with the extended dataset and further compare and analyze the final results.